# Load dataset

In [1]:
import sys, os
sys.path.append('..')
import numpy as np
from imutils import paths



from utils.preprocessors.aspect_aware_preprocessor import AspectAwarePreprocessor
from utils.preprocessors.image_to_array_preprocessor import ImageToArrayPreprocessor
from utils.io.simple_dataset_loader import SimpleDatasetLoader


dataset_path = '../datasets/plantVillage-test-mohanty'

# Get the list of image names
image_paths = list(paths.list_images(dataset_path))

print(len(image_paths), 'images loaded')

# Get unique class_names
class_names = [pt.split(os.path.sep)[-2] for pt in image_paths]
class_names = [str(x) for x in np.unique(class_names)]
print(class_names)


# Preprocessors
aap = AspectAwarePreprocessor(227, 227)
itap = ImageToArrayPreprocessor()

# Add preprocessors to DataSetLoader class
sdl = SimpleDatasetLoader(preprocessors=[aap, itap])

(data, labels) = sdl.load(image_paths, verbose=500)
data = data.astype('float') / 255.0



Using TensorFlow backend.
2298 images loaded
['Tomato___Bacterial_spot', 'Tomato___Early_blight', 'Tomato___healthy']
[INFO]: Processed 500/2298
[INFO]: Processed 1000/2298
[INFO]: Processed 1500/2298
[INFO]: Processed 2000/2298


# Encode labels and split dataset

In [3]:
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# Split data into training (75%) and testing (25%) data
(train_x, test_x, train_y, test_y) = train_test_split(data, labels,
test_size=0.25, random_state=42)

# Convert the labels from integers to vectors
lb = LabelBinarizer()
train_y = lb.fit_transform(train_y)
test_y = lb.transform(test_y)

[[0 1 0]
 [0 0 1]
 [1 0 0]
 ...
 [0 1 0]
 [0 0 1]
 [1 0 0]]


# Initialize model

In [1]:
import sys, os
sys.path.append('..')

from utils.models.alexnet import alexnet
from keras.utils import plot_model
from keras.optimizers import SGD

model = alexnet(len(class_names))
model.summary()
plot_model(model, to_file="alexnet_model.png", show_shapes=True)

# copile model
opt = SGD(lr=0.05)
model.compile(loss="categorical_crossentropy", optimizer=opt,
              metrics=["accuracy"])



Using TensorFlow backend.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 55, 55, 96)        34944     
_________________________________________________________________
activation_1 (Activation)    (None, 55, 55, 96)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 27, 27, 96)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 17, 17, 256)       2973952   
_________________________________________________________________
activation_2 (Activation)    (None, 17, 17, 256)       0         
_____________________________________________________________

# Fit model

In [4]:
# train the network
print("[INFO] training network...")
H = model.fit(train_x, train_y, validation_data=(test_x, test_y),
              batch_size=32, epochs=100, verbose=1)

model.save('alexnet_model.h5')

[INFO] training network...
Instructions for updating:
Use tf.cast instead.
Train on 3538 samples, validate on 1180 samples
Epoch 1/100
1888/3538 [===============>..............] - ETA: 2:30 - loss: 1.0615 - acc: 0.4603

# Generate classfication report

In [0]:
from sklearn.metrics import classification_report


# If one hot encoding is used
print(classification_report(test_y, model.predict(test_x),
                            target_names=classNames))


# Plot model

In [0]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, 100), H.history["loss"], label="train_loss")
plt.plot(np.arange(0, 100), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, 100), H.history["acc"], label="train_acc")
plt.plot(np.arange(0, 100), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend()
plt.savefig(args["output"])
plt.show()